In [1]:
import pandas as pd
import numpy as np
import re 
import os
import time
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.cross_validation import LabelKFold
from sklearn.grid_search import GridSearchCV
import seaborn as sns
%matplotlib inline

C:\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
submit = pd.read_csv('submit_model_j005.csv')
dataset = pd.read_csv('jorge_dataset_modified.csv')

In [3]:
submit.head()

,id,Demanda_uni_equil
0,924190,3.549768
1,4521987,3.368267
2,3448837,11.458047
3,6264169,8.020918
4,6929368,5.646762


In [4]:
updates = dataset[dataset.Semana == 10].set_index('id')
submit = submit.set_index('id')
updates.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin,num_prod,num_prod_uni
id,,,,,,,,,,,,,,,,,,,,
906304,1110,7,15766,0.0,35651,3301,10,2008,2.079442,2.128370,2,2.079442,2.484907,1.386294,1.791759,7.742402,2,1,10,10
970784,1110,7,15766,0.0,3894,3301,10,2008,2.639057,1.852653,2,2.639057,1.098612,1.609438,1.609438,6.956545,14,1,10,10
1027965,1110,7,15766,0.0,35452,3301,10,2008,1.609438,2.235116,2,1.609438,1.945910,1.609438,0.000000,5.164786,14,1,10,10
3185929,1110,7,15766,0.0,41938,3301,10,2008,1.386294,1.556816,2,1.386294,1.098612,1.386294,0.000000,3.871201,2,1,10,10
3236895,1110,7,15766,0.0,37057,3301,10,2008,2.079442,1.758743,2,2.079442,1.609438,2.197225,1.386294,7.272398,14,1,10,10


In [5]:
updates.Demanda_uni_equil = submit.Demanda_uni_equil.apply(lambda x: np.log1p(x))
updates.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin,num_prod,num_prod_uni
id,,,,,,,,,,,,,,,,,,,,
906304,1110,7,15766,1.918475,35651,3301,10,2008,2.079442,2.128370,2,2.079442,2.484907,1.386294,1.791759,7.742402,2,1,10,10
970784,1110,7,15766,1.845054,3894,3301,10,2008,2.639057,1.852653,2,2.639057,1.098612,1.609438,1.609438,6.956545,14,1,10,10
1027965,1110,7,15766,1.690806,35452,3301,10,2008,1.609438,2.235116,2,1.609438,1.945910,1.609438,0.000000,5.164786,14,1,10,10
3185929,1110,7,15766,1.325323,41938,3301,10,2008,1.386294,1.556816,2,1.386294,1.098612,1.386294,0.000000,3.871201,2,1,10,10
3236895,1110,7,15766,1.824628,37057,3301,10,2008,2.079442,1.758743,2,2.079442,1.609438,2.197225,1.386294,7.272398,14,1,10,10


In [6]:
updates = updates.reset_index()
dataset[dataset.Semana == 10] = updates


In [7]:
dataset = dataset.fillna(0)
dataset[dataset.Semana == 10].head()


,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,...,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin,num_prod,num_prod_uni
160,1110.0,7.0,1914789.0,1.736640,5310.0,3301.0,10.0,2008.0,1308192.0,1.945910,...,2.0,1.945910,0.000000,0.000000,0.000000,1.945910,14.0,1.0,3.0,3.0
162,1110.0,7.0,1916494.0,2.960465,35141.0,3301.0,10.0,2008.0,5139756.0,2.772589,...,2.0,2.772589,2.772589,2.995732,3.258097,11.799006,32.0,1.0,1.0,1.0
163,1110.0,7.0,2251340.0,3.931335,202.0,3301.0,10.0,2008.0,4834776.0,3.713572,...,2.0,3.713572,3.931826,4.025352,4.110874,15.781623,25.0,1.0,1.0,1.0
164,1110.0,7.0,2251341.0,1.008421,145.0,3301.0,10.0,2008.0,4918641.0,1.098612,...,2.0,1.098612,0.693147,0.693147,0.000000,2.484907,32.0,1.0,1.0,1.0
165,1110.0,7.0,2331717.0,0.000000,1216.0,3301.0,10.0,2008.0,568789.0,0.693147,...,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,1.0,5.0,5.0


In [8]:
dataset.to_csv('jorge_dataset_modifiedwPred.csv')